In [1]:
%load_ext autoreload

%autoreload 2

import gauleg as gl 
import sympy as sp 
import numpy as np 
import pandas as pd 
import math

## Finding Lm Ln Polynomials 


In [21]:
def legendre_polynomial(n):
    y = sp.symbols('y')
    if n == 0:
        return sp.Lambda(y, 1)
    elif n == 1:
        return sp.Lambda(y, y)
    else:
        P_n_minus_1 = legendre_polynomial(n-1)
        P_n_minus_2 = legendre_polynomial(n-2)
        return sp.Lambda(y, ((2*n - 1) * y * P_n_minus_1(y) - (n - 1) * P_n_minus_2(y)) / n)

# Example usage:
n = 4
Ln = legendre_polynomial(n)
Ln


Lambda(y, -9*y**2/8 + 7*y*(5*y*(3*y**2/2 - 1/2)/3 - 2*y/3)/4 + 3/8)

## Computing Anm using GL Quadrature --> finding A 

In [23]:
n = [0,1,2,3,4]
m = [0,1,2,3,4]

def Anm(n, m):
    y = sp.symbols('y')
    Ln = legendre_polynomial(n)
    Lm = legendre_polynomial(m)
    return y * Ln(y) * Lm(y)

def Anm_gauleg(xi, ci, b , a, n2, n, m):
    sum = 0 
    Anm_expr = Anm(n,m)
    for i in range(n2):
        y_value = (0.5*(((b-a)*xi[i])+(b+a)))
        prod = ci[i]* Anm_expr.subs(sp.symbols('y'), y_value)
        sum = sum + prod 
    return sum 

def A_matrix(n,m):
    A_matrix = sp.zeros(len(n),len(m))
    n2 = math.floor((len(n)+len(m))/2) + 1
    xi = gl.gauleg(n2)[0] # satisfying dop of GL for p+1 is 2p + 1
    ci = gl.gauleg(n2)[1]
    for ni in n:
        for mi in m:
            A_matrix[ni ,mi] = Anm_gauleg(xi= xi, ci = ci, b = 1, a = -1, n2 = n2, n = ni, m = mi)
    return A_matrix

A = A_matrix(n,m)
A



Matrix([
[-2.77555756156289e-17,    0.666666666666667,                    0, 3.19189119579733e-16,                    0],
[    0.666666666666667,                    0,    0.266666666666667,                    0, 2.63677968348475e-16],
[                    0,    0.266666666666667,                    0,    0.171428571428572, 6.93889390390723e-18],
[ 3.19189119579733e-16,                    0,    0.171428571428572,                    0,    0.126984126984127],
[                    0, 2.63677968348475e-16, 6.93889390390723e-18,    0.126984126984127,                    0]])

## Define A(x,y) 

In [24]:
def a(x, y):
    x = sp.symbols('x')
    y = sp.symbols('y')
    return 5 * x + 1 + y * sp.sin(x)

def a0(x):
    x = sp.symbols('x')
    return 5 * x + 1 

def a1(x):
    x = sp.symbols('x')
    return 0



## Define (S1) matrix and (S0) matrix


In [27]:
def GL(xi, ci, y, x, n2, func):
    sum = 0 
    if isinstance(func, int):
        final = 0 
    else:
        for i in range(n2):
            x_value = 0.5 * ((y - x) * xi[i] + (y + x))
            prod = ci[i] * func.subs(sp.symbols('x'), x_value)
            sum += prod 
        final = (y - x) / 2 * sum
    return final




def phiij(numofnodes, i, j, l, xlist, func):
	finalsum = 0
	for k in range(numofnodes-1):

		x = xlist[k]
		y = xlist[k+1]
	 
		if k == i :
			d_phi_i = 2**l 
		elif k == i + 1 :
			d_phi_i = -2**l 
		else: 
			d_phi_i = 0 
 
		if k == j : 
			d_phi_j = 2**l 
		elif k == j + 1 :
			d_phi_j = -2**l 
		else:
			d_phi_j = 0 

		n2 = 5 
		xi = gl.gauleg(n2)[0]
		ci = gl.gauleg(n2)[1]
		onegl = GL(xi = xi, ci = ci, y = y, x = x, n2 = 5, func = func) * d_phi_i *d_phi_j
		finalsum = finalsum + onegl

	return finalsum

def S1(listi, listj, func):
    S1 = sp.zeros(len(listi), len(listj))
    target = func
    for i in listi :
        for j in listj:
            S1[i,j] = phiij(numofnodes=2**l +1, i = i, j = j, l = l, xlist = gl.listi(a= 0,b =1, h = 2 **(-l), n1 = 2 **l + 1), func = target)
    return S1


l = 4
ijlist = list(range(2**l-1))
S1(listi = ijlist, listj=ijlist, func= a1(0))


Matrix([
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [28]:
# compute S0 

def S0(listi, listj, func):
    S0 = sp.zeros(len(listi), len(listj))
    target = func
    for i in listi:
        for j in listj:
            S0[i,j] = phiij(numofnodes=2**l +1, i = i, j = j, l = l, xlist = gl.listi(a= 0,b =1, h = 2 **(-l), n1 = 2 **l + 1), func = target)

    return S0 

S0(listi=ijlist, listj=ijlist, func = a0(0))

Matrix([
[ 42.0, -23.5,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
[-23.5,  52.0, -28.5,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
[    0, -28.5,  62.0, -33.5,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
[    0,     0, -33.5,  72.0, -38.5,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
[    0,     0,     0, -38.5,  82.0, -43.5,     0,     0,     0,     0,     0,     0,     0,     0,     0],
[    0,     0,     0,     0, -43.5,  92.0, -48.5,     0,     0,     0,     0,     0,     0,     0,     0],
[    0,     0,     0,     0,     0, -48.5, 102.0, -53.5,     0,     0,     0,     0,     0,     0,     0],
[    0,     0,     0,     0,     0,     0, -53.5, 112.0, -58.5,     0,     0,     0,     0,     0,     0],
[    0,     0,     0,     0,     0,     0,     0, -58.5, 122.0, -63.5,     0,     0,     0,     0,     0],
[    0,     0,     0,     0,

## Define Kroneker Notation 


In [29]:
# n, m are integers, probably need to loop it for every possible n, m 

def delta_mn(m,n):
    if n == m:
        return 1 
    else:
        return 0 

def delta(n,m):
    delta = sp.zeros(len(n), len(m))
    for ni in n:
        for mi in m: 
            delta[ni,mi] = delta_mn(ni,mi)

    return delta 

delta(n,m)

Matrix([
[1, 0, 0, 0, 0],
[0, 1, 0, 0, 0],
[0, 0, 1, 0, 0],
[0, 0, 0, 1, 0],
[0, 0, 0, 0, 1]])

## Kron Matrix A and S1 

In [30]:
def A_S1(matrixA, matrixS1):
    A_S1 = sp.kronecker_product(matrixA, matrixS1)
    return A_S1

A_S1(matrixA= A_matrix(n= n, m= m), matrixS1= S1(listi=ijlist, listj=ijlist, func = a1(0)))


Matrix([
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

## Define S0 * delta_mn


In [31]:
def S0_delta(matrixS0, matrixdelta):
    return sp.kronecker_product(matrixS0 , matrixdelta) 

S0_delta(matrixS0=S0(listi=ijlist, listj=ijlist, func = a0(0)), matrixdelta=delta(n,m))

Matrix([
[ 42.0,     0,     0,     0,     0, -23.5,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
[    0,  42.0,     0,     0,     0,     0, -23.5,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,  

## Define S(in)(jm)


In [32]:
def S_in_jm(matrixS1_Amn, matrixS0_delta):
    return matrixS0_delta + matrixS1_Amn

S_in_jm(matrixS1_Amn= A_S1(matrixA=A_matrix(n,m), matrixS1=S1(listi = ijlist, listj=ijlist, func= a1(0))), matrixS0_delta=S0_delta(matrixS0=S0(listi=ijlist, listj=ijlist, func = a0(0)), matrixdelta=delta(n,m)))

Matrix([
[ 42.0,     0,     0,     0,     0, -23.5,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
[    0,  42.0,     0,     0,     0,     0, -23.5,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,  

## Define Fi and F 

In [33]:
def intergrand(numofnodes, i, l, xlist, f, ci, xi, n2):
    def phi_j(numofnodes, i, l, k):
        def phi_j_function(s):
            if k == i:
                return 2**l * (s - (i / 2**l))
            elif k == i + 1:
                return -(2**l * (s - ((i + 2) / 2**l)))
            else:
                return 0
        return phi_j_function

    finalsum = 0
    for k in range(numofnodes-1):
        phi_j_func = phi_j(numofnodes, i, l, k)
        x = xlist[k]
        y = xlist[k+1]
        sum = 0
        for j in range(n2):
            s = 0.5 * ((y - x) * xi[j] + (y + x))
            h = f(s) * phi_j_func(s)
            prod = ci[j] * h
            sum += prod
        onegl = (y - x) / 2 * sum
        finalsum += onegl

    return finalsum

def f(s): ## is f(s) supposed to be a function of y? --> u(x,y) and a(x,y) = f(x,y)
    return -4 + math.cos(s) -2*s*math.cos(s) - 2* math.sin(s)

def F(n, i):
    F = sp.zeros((len(i)*len(n)), 1)
    for ii in i :
        for ni in n: 
            if ni == 0:
                F[(ii*ni), 0] = intergrand(numofnodes=2**l +1, i = ii, l = l, xlist =gl.listi(a= 0,b =1, h = 2 **(-l), n1 = 2 **l + 1), f = f, ci = gl.gauleg(n2 = 5)[1], xi = gl.gauleg(n2=5)[0], n2 = 5)
            else: 
                F[(ii*ni), 0] = 0 
    return F

F(n, i = ijlist)



Matrix([
[-0.383015567460435],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[                 0],
[

## Solving for C 

In [34]:
def C(matrixS, matrixF):
    S_inv = matrixS.inv()
    C = S_inv * matrixF
    return C

C( matrixS= S_in_jm(matrixS1_Amn= A_S1(matrixA=A_matrix(n,m), matrixS1=S1(listi = ijlist, listj=ijlist, func= a1(0))), matrixS0_delta=S0_delta(matrixS0=S0(listi=ijlist, listj=ijlist, func = a0(0)), matrixdelta=delta(n,m))), matrixF= F(n, i = ijlist) ) 

Matrix([
[  -0.0175738125076829],
[                    0],
[                    0],
[                    0],
[                    0],
[  -0.0151099811856276],
[                    0],
[                    0],
[                    0],
[                    0],
[  -0.0130784009727048],
[                    0],
[                    0],
[                    0],
[                    0],
[  -0.0113500416870839],
[                    0],
[                    0],
[                    0],
[                    0],
[ -0.00984614464634878],
[                    0],
[                    0],
[                    0],
[                    0],
[ -0.00851510933443382],
[                    0],
[                    0],
[                    0],
[                    0],
[ -0.00732129415776783],
[                    0],
[                    0],
[                    0],
[                    0],
[ -0.00623905049293979],
[                    0],
[                    0],
[                    0],
[               

## Computing error 

In [35]:
def approx_new(ijlist, c, F):
    approx_new = 0 
    for i in ijlist : 
        prod = c[i] * F[i]
        approx_new = approx_new + prod
    return approx_new

approx_new(ijlist= ijlist, c = C( matrixS= S_in_jm(matrixS1_Amn= A_S1(matrixA=A_matrix(n,m), matrixS1=S1(listi = ijlist, listj=ijlist, func= a1(0))), matrixS0_delta=S0_delta(matrixS0=S0(listi=ijlist, listj=ijlist, func = a0(0)), matrixdelta=delta(n,m))), matrixF= F(n, i = ijlist) ) , F =F(n, i = ijlist) )

0.00673104377007347

Therefore, We have shown that as n, m, l increases, the error decreases 
